# Concatinating .csv Files and Creating New df_subset with Improved Column Names

In [ ]:
import csv
import glob
import pandas as pd

# Display all columns
pd.set_option('display.max_columns', None)

# Display all rows
pd.set_option('display.max_rows', None)

# Reads all csv files in this folder and concatenates them
csv_files = glob.glob('*.csv')
df_list = [pd.read_csv(file) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)

# Dictionary of column names to change
cols_to_rename = {
    "tourney_id": "tournament_id",
    "tourney_name": "tournament_name",
    "tourney_date": "tournament_date",
    "winner_ht": "p1_height",
    "winner_id": "p1_id",
    "winner_name": "p1_name",
    "winner_age": "p1_age",
    "loser_age": "p2_age", 
    "winner_hand": "p1_hand",
    "loser_ht": "p2_height",
    "loser_id": "p2_id",
    "loser_name": "p2_name",
    "loser_hand": "p2_hand",
    "tourney_level": "tournament_level",
}

# Rename useful columns
df.rename(columns=cols_to_rename, inplace=True)

# remove Round Robin (RR) and Bronze medal (BR) round rows
df = df.loc[~df['round'].isin(['RR', 'BR'])].copy()

# Change format of tournament date column
df['tournament_date'] = pd.to_datetime(df['tournament_date'], format='%Y%m%d')

# Define round order
round_order = ["Q1", "Q2", "Q3", "R128", "R64", "R32", "R16", "QF", "SF", "F"]

# Create an ordered categorical column for tournament round
df["round"] = pd.Categorical(df["round"], categories=round_order, ordered=True)

# Build the list of sort keys that are actually present, then sort
sort_keys = [c for c in ["tournament_date", "tournament_id", "round"] if c in df.columns]
df = df.sort_values(sort_keys, kind="mergesort").reset_index(drop=True)

In [ ]:
# Create subset of dataframe with useful columns
df_subset = df[['tournament_date','tournament_name', 'tournament_level', 'surface', 'p1_name', 'p1_id', 'p1_age', 'p1_height', 'p1_hand', 'p2_name', 'p2_id', 'p2_age', 'p2_height', 'p2_hand']]
df_subset = df_subset.copy()

# Cleaning Data

### Looking for Anomalous Data

In [ ]:
# Displays number of rows in a dataframe
len(df_subset) 

In [ ]:
# Displayes number of NaN values for each column
df_subset.isna().sum()

In [ ]:
df_subset.describe()

## Anomolous Surface Data

In [ ]:
df_subset[df_subset['surface'].isna()]

###### There are no rows with missing values in the surface column.

## Anomolous Age Data

In [ ]:
df_subset[(df_subset['p1_age'].isna()) | (df_subset['p2_age'].isna())]

###### All of the players that have missing p1_age and p2_age values also have missing height data. I can not find these player's ages or heights on the ATP website. There are a total of 115 missing age entries in a data frame of 143272 so these will be removed from the df_subset.

In [ ]:
df_subset['p1_age'].describe()

In [ ]:
df_subset['p2_age'].describe()

In [ ]:
df_subset[(df_subset['p1_age'] < 16) | (df_subset['p2_age'] < 16)]

In [ ]:
df_subset[(df_subset['p1_age'] > 45) | (df_subset['p2_age'] > 45)]

###### I have checked for anomalously old and young players. All of the very young and old players listed in the dataset have their correct ages listed. 

## Anomolous Height Data

In [ ]:
# Total number of rows with missing height data for eiter p1 or p2 
len(df_subset[(df_subset['p1_height'].isna()) | (df_subset['p2_height'].isna())])

###### There are 17296 rows in the dataset with missing height data.

In [ ]:
# Create list of players with missing height data
missing_height_p1_names = df_subset[df_subset['p1_height'].isna()]['p1_name']
missing_height_p2_names = df_subset[df_subset['p2_height'].isna()]['p2_name']
missing_height_names = set(missing_height_p1_names) | set(missing_height_p2_names)

In [ ]:
import wptools
import re

def get_heights(names): 
    # store results as name: height_cm
    heights = {}  
    
    for name in names:
        try:
            page = wptools.page(name, silent=True).get_parse() 
            infobox = page.data.get('infobox', {}) 
            height_raw = infobox.get('height', '') 
            
            # Extract height in meters from the wikipedia page
            match = re.search(r'\{\{height\|m\|=\|([\d.]+)\}\}', height_raw) 
            if match: 
                height_cm = float(match.group(1)) * 100
                heights[name] = height_cm
            else:
                # If it could not be parsed
                heights[name] = None  
        except Exception:
            # page not found or other error
            heights[name] = None  
    
    return heights

#get_heights(missing_height_names)

###### I have searched wikipedia for their height data to amend the entries, but these players do not have wikipedia pages. The NaN results will not show in later in seaborn plots, however these results will cause errors later on when trying to train a model on this data so they will be removed. There are 17296 rows being deleted which may impact ELO calculations, therefore I will remove these rows after the ELOs have been calculated.    

In [ ]:
df_subset['p1_height'].describe()

In [ ]:
df_subset['p2_height'].describe()

###### In both p1 and p2 height columns, there is a player listed as being 3 cm tall, this is clearly anomolous data so I will find their names and try to amend their height data.

In [ ]:
# Checking for anomolously short players
df_subset[(df_subset['p1_height'] <= 160) | (df_subset['p2_height'] <= 160)]

In [ ]:
# Checking for anomolously tall players
df_subset[(df_subset['p1_height'] >= 211) | (df_subset['p2_height'] >= 211)]

###### I looked up the players with anonomous height data i.e. height = 3 cm, their heights are not shown on the ATP website so they will be removed after calculating ELOs etc.

## Missing Player Hand Data

In [ ]:
df_subset[(df_subset['p1_hand'] == 'U') | (df_subset['p2_hand'] == 'U')].describe()

###### Players with unknown handedness usually have other missing data so they will be removed after calculating ELOs etc.

# Calculating New Features for Dataframe

## Calculating Age Differences

In [ ]:
# Create empty lists to store age differences
p1_age_diff_list = []
p2_age_diff_list = []

# loops over the age columns and calculates players age difference
for p1, p2 in zip(df_subset['p1_age'], df_subset['p2_age']):
    p1_age_diff = p1 - p2
    p2_age_diff = p2 - p1

    # Adds calculated age differences to list
    p1_age_diff_list.append(p1_age_diff)
    p2_age_diff_list.append(p2_age_diff)

# Assign lists to new df columns
df_subset['p1_age_diff'] = p1_age_diff_list
df_subset['p2_age_diff'] = p2_age_diff_list
df_subset['p1_age_diff'] = df_subset['p1_age_diff'].round(1)
df_subset['p2_age_diff'] = df_subset['p2_age_diff'].round(1)

## Calculating Height Differences

In [ ]:
# Create empty lists to store height differences
p1_height_diff_list = []
p2_height_diff_list = []

# loops over the height columns and calculates players height difference
for p1, p2 in zip(df_subset['p1_height'], df_subset['p2_height']):
    p1_height_diff = p1 - p2
    p2_height_diff = p2 - p1

    # Adds calculated age differences to list
    p1_height_diff_list.append(p1_height_diff)
    p2_height_diff_list.append(p2_height_diff)

# Assign lists to new df columns
df_subset['p1_height_diff'] = p1_height_diff_list
df_subset['p2_height_diff'] = p2_height_diff_list

# Calculating Previous H2H Wins Against Opponent Columns

In [ ]:
from collections import defaultdict 

# Dictionary that stores head-to-head match results. Returns 0 if the key does not exist (players have never played before)
h2h_wins_dict = defaultdict(int)

# Dictionary to store sequence of winners of matches between players
h2h_history_dict = defaultdict(list)

# Stores total h2h wins before the current match
p1_h2h_wins_before = []
p2_h2h_wins_before = []

# Stores total h2h win difference before the current match
p1_h2h_wins_total_diff_before = []
p2_h2h_wins_total_diff_before = []

# Stores h2h win difference in last game
p1_h2h_wins_last1_diff_before = []
p2_h2h_wins_last1_diff_before = []

# Stores h2h win difference in last 2 games
p1_h2h_wins_last2_diff_before = []
p2_h2h_wins_last2_diff_before = []

# Stores h2h win difference in last 3 games
p1_h2h_wins_last3_diff_before = []
p2_h2h_wins_last3_diff_before = []

# Stores h2h win difference in last 4 games
p1_h2h_wins_last4_diff_before = []
p2_h2h_wins_last4_diff_before = []

# Stores h2h win difference in last 5 games
p1_h2h_wins_last5_diff_before = []
p2_h2h_wins_last5_diff_before = []

# Stores h2h win difference in last 10 games
p1_h2h_wins_last10_diff_before = []
p2_h2h_wins_last10_diff_before = []


# Iterate through each match in df_subset, returning player_ids as pairs  
for p1, p2 in zip(df_subset['p1_id'], df_subset['p2_id']):
    
    # Creates keys for head-to-head matches
    wins_key1 = (p1, p2)
    wins_key2 = (p2, p1)
    wins_match_key = tuple(sorted([p1, p2]))

    # Get total wins for each player before this match
    p1_h2h_wins = h2h_wins_dict[wins_key1]
    p2_h2h_wins = h2h_wins_dict[wins_key2]
    
    # Saves wins to respective p1_h2h_wins_before and p2_h2h_wins_before lists to then be used for dataframe columns
    p1_h2h_wins_before.append(p1_h2h_wins)
    p2_h2h_wins_before.append(p2_h2h_wins)
    
    # Calculates wins difference
    p1_h2h_wins_diff = p1_h2h_wins - p2_h2h_wins
    p2_h2h_wins_diff = p2_h2h_wins - p1_h2h_wins
    
    # Saves differences to px_h2h_wins_diff_before lists to then be used for dataframe columns
    p1_h2h_wins_total_diff_before.append(p1_h2h_wins_diff)
    p2_h2h_wins_total_diff_before.append(p2_h2h_wins_diff)

    # Get last 1 match results and compute difference
    history_last1 = h2h_history_dict[wins_match_key][-1:]
    p1_last1_wins = history_last1.count(p1)
    p2_last1_wins = history_last1.count(p2)
    p1_last1_diff = p1_last1_wins - p2_last1_wins
    p2_last1_diff = p2_last1_wins - p1_last1_wins
    p1_h2h_wins_last1_diff_before.append(p1_last1_diff)
    p2_h2h_wins_last1_diff_before.append(p2_last1_diff)

    # Get last 2 match results and compute difference
    history_last2 = h2h_history_dict[wins_match_key][-2:]
    p1_last2_wins = history_last2.count(p1)
    p2_last2_wins = history_last2.count(p2)
    p1_last2_diff = p1_last2_wins - p2_last2_wins
    p2_last2_diff = p2_last2_wins - p1_last2_wins
    p1_h2h_wins_last2_diff_before.append(p1_last2_diff)
    p2_h2h_wins_last2_diff_before.append(p2_last2_diff)

    # Get last 3 match results and compute difference
    history_last3 = h2h_history_dict[wins_match_key][-3:]
    p1_last3_wins = history_last3.count(p1)
    p2_last3_wins = history_last3.count(p2)
    p1_last3_diff = p1_last3_wins - p2_last3_wins
    p2_last3_diff = p2_last3_wins - p1_last3_wins
    p1_h2h_wins_last3_diff_before.append(p1_last3_diff)
    p2_h2h_wins_last3_diff_before.append(p2_last3_diff)

    # Get last 4 match results and compute difference
    history_last4 = h2h_history_dict[wins_match_key][-4:]
    p1_last4_wins = history_last4.count(p1)
    p2_last4_wins = history_last4.count(p2)
    p1_last4_diff = p1_last4_wins - p2_last4_wins
    p2_last4_diff = p2_last4_wins - p1_last4_wins
    p1_h2h_wins_last4_diff_before.append(p1_last4_diff)
    p2_h2h_wins_last4_diff_before.append(p2_last4_diff)

    # Get last 5 match results and compute difference
    history_last5 = h2h_history_dict[wins_match_key][-5:]
    p1_last5_wins = history_last5.count(p1)
    p2_last5_wins = history_last5.count(p2)
    p1_last5_diff = p1_last5_wins - p2_last5_wins
    p2_last5_diff = p2_last5_wins - p1_last5_wins
    p1_h2h_wins_last5_diff_before.append(p1_last5_diff)
    p2_h2h_wins_last5_diff_before.append(p2_last5_diff)

    # Get last 10 match results and compute difference
    history_last10 = h2h_history_dict[wins_match_key][-10:]
    p1_last10_wins = history_last10.count(p1)
    p2_last10_wins = history_last10.count(p2)
    p1_last10_diff = p1_last10_wins - p2_last10_wins
    p2_last10_diff = p2_last10_wins - p1_last10_wins
    p1_h2h_wins_last10_diff_before.append(p1_last10_diff)
    p2_h2h_wins_last10_diff_before.append(p2_last10_diff)

    # Player 1 always wins in this df_subset, updates head-to-head
    h2h_wins_dict[wins_key1] += 1  

    # Updates h2h history dicitonary 
    h2h_history_dict[wins_match_key].append(p1)

# Assign to dataframe
df_subset['p1_h2h_wins'] = p1_h2h_wins_before
df_subset['p2_h2h_wins'] = p2_h2h_wins_before
df_subset['p1_h2h_wins_before_total_diff'] = p1_h2h_wins_total_diff_before
df_subset['p2_h2h_wins_before_total_diff'] = p2_h2h_wins_total_diff_before
df_subset['p1_h2h_wins_before_last1_diff'] = p1_h2h_wins_last1_diff_before
df_subset['p2_h2h_wins_before_last1_diff'] = p2_h2h_wins_last1_diff_before
df_subset['p1_h2h_wins_before_last2_diff'] = p1_h2h_wins_last2_diff_before
df_subset['p2_h2h_wins_before_last2_diff'] = p2_h2h_wins_last2_diff_before
df_subset['p1_h2h_wins_before_last3_diff'] = p1_h2h_wins_last3_diff_before
df_subset['p2_h2h_wins_before_last3_diff'] = p2_h2h_wins_last3_diff_before
df_subset['p1_h2h_wins_before_last4_diff'] = p1_h2h_wins_last4_diff_before
df_subset['p2_h2h_wins_before_last4_diff'] = p2_h2h_wins_last4_diff_before
df_subset['p1_h2h_wins_before_last5_diff'] = p1_h2h_wins_last5_diff_before
df_subset['p2_h2h_wins_before_last5_diff'] = p2_h2h_wins_last5_diff_before
df_subset['p1_h2h_wins_before_last10_diff'] = p1_h2h_wins_last10_diff_before
df_subset['p2_h2h_wins_before_last10_diff'] = p2_h2h_wins_last10_diff_before

# Calculating ELO

In [ ]:
from collections import defaultdict

# Dictionary that stores players ELO. Returns 1500 if the key does not exist (player has not played before)
elo_dict = defaultdict(lambda: 1500)

# Stores immediate elo
p1_elo_before = []
p2_elo_before = []

# Iterate through each match in df_subset, returning player_ids as pairs 
for p1, p2 in zip(df_subset['p1_id'], df_subset['p2_id']):
    elo_key1 = p1
    elo_key2 = p2

    # Checks dictionary for ELO and stores the ELO as px_elo
    p1_elo = elo_dict[elo_key1]
    p2_elo = elo_dict[elo_key2]

    # Adds ELO to the list px_elo_before
    p1_elo_before.append(p1_elo)
    p2_elo_before.append(p2_elo)

    # Calculates expected score
    p1_expected_score = 1 / (1 + 10**((p2_elo - p1_elo)/400))
    p2_expected_score = 1 / (1 + 10**((p1_elo - p2_elo)/400))

    # Calculates ELO after the match
    K = 32
    p1_elo_after = int(p1_elo + K * (1 - p1_expected_score))
    p2_elo_after = int(p2_elo + K * (0 - p2_expected_score))

    # Stores new ELO in the dictionary
    elo_dict[elo_key1] = p1_elo_after
    elo_dict[elo_key2] = p2_elo_after
    
# Assign to dataframe
df_subset['p1_elo_before'] = p1_elo_before
df_subset['p2_elo_before'] = p2_elo_before

#### Calculating Rolling ELO

In [ ]:
df_subset['p1_elo_rolling_last5'] = (df_subset.groupby('p1_id')['p1_elo_before'].transform(lambda x: x.shift().rolling(5, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p2_elo_rolling_last5'] = (df_subset.groupby('p2_id')['p2_elo_before'].transform(lambda x: x.shift().rolling(5, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p1_elo_rolling_last10'] = (df_subset.groupby('p1_id')['p1_elo_before'].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p2_elo_rolling_last10'] = (df_subset.groupby('p2_id')['p2_elo_before'].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p1_elo_rolling_last20'] = (df_subset.groupby('p1_id')['p1_elo_before'].transform(lambda x: x.shift().rolling(20, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p2_elo_rolling_last20'] = (df_subset.groupby('p2_id')['p2_elo_before'].transform(lambda x: x.shift().rolling(20, min_periods=1).mean())).fillna(1500).astype(int)

#### Calculating Rolling ELO difference

In [ ]:
# Create empty lists to store rolling ELO differences
p1_elo_rolling_last5_diff_list = []
p2_elo_rolling_last5_diff_list = []

p1_elo_rolling_last10_diff_list = []
p2_elo_rolling_last10_diff_list = []

p1_elo_rolling_last20_diff_list = []
p2_elo_rolling_last20_diff_list = []

# Loops over the last5 rolling ELO columns and calculates the players ELO difference
for p1, p2 in zip(df_subset['p1_elo_rolling_last5'], df_subset['p2_elo_rolling_last5']):
    p1_elo_rolling_last5_diff = p1 - p2
    p2_elo_rolling_last5_diff = p2 - p1

    # Adds calculated ELO differences to list
    p1_elo_rolling_last5_diff_list.append(p1_elo_rolling_last5_diff)
    p2_elo_rolling_last5_diff_list.append(p2_elo_rolling_last5_diff)

# Loops over the last10 rolling ELO columns and calculates the players ELO difference
for p1, p2 in zip(df_subset['p1_elo_rolling_last10'], df_subset['p2_elo_rolling_last10']):
    p1_elo_rolling_last10_diff = p1 - p2
    p2_elo_rolling_last10_diff = p2 - p1

    # Adds calculated ELO differences to list
    p1_elo_rolling_last10_diff_list.append(p1_elo_rolling_last10_diff)
    p2_elo_rolling_last10_diff_list.append(p2_elo_rolling_last10_diff)

# Loops over the last20 rolling ELO columns and calculates the players ELO difference
for p1, p2 in zip(df_subset['p1_elo_rolling_last20'], df_subset['p2_elo_rolling_last20']):
    p1_elo_rolling_last20_diff = p1 - p2
    p2_elo_rolling_last20_diff = p2 - p1

    # Adds calculated ELO differences to list
    p1_elo_rolling_last20_diff_list.append(p1_elo_rolling_last20_diff)
    p2_elo_rolling_last20_diff_list.append(p2_elo_rolling_last20_diff)


# Assign lists to new df columns
df_subset['p1_elo_rolling_last5_diff_before'] = p1_elo_rolling_last5_diff_list
df_subset['p2_elo_rolling_last5_diff_before'] = p2_elo_rolling_last5_diff_list
df_subset['p1_elo_rolling_last10_diff_before'] = p1_elo_rolling_last10_diff_list
df_subset['p2_elo_rolling_last10_diff_before'] = p2_elo_rolling_last10_diff_list
df_subset['p1_elo_rolling_last20_diff_before'] = p1_elo_rolling_last20_diff_list
df_subset['p2_elo_rolling_last20_diff_before'] = p2_elo_rolling_last20_diff_list

#### Calculating ELO difference

In [ ]:
# Create empty lists to store ELO differences
p1_elo_diff_list = []
p2_elo_diff_list = []

# Loops over the ELO columns and calculates the players ELO difference
for p1, p2 in zip(df_subset['p1_elo_before'], df_subset['p2_elo_before']):
    p1_elo_diff = p1 - p2
    p2_elo_diff = p2 - p1

    # Adds calculated ELO differences to list
    p1_elo_diff_list.append(p1_elo_diff)
    p2_elo_diff_list.append(p2_elo_diff)

# Assign lists to new df columns
df_subset['p1_elo_diff_before'] = p1_elo_diff_list
df_subset['p2_elo_diff_before'] = p2_elo_diff_list

#### Displays Total Number of Games Played on each Surface

In [ ]:
surface_dict = {}

for surface in df_subset['surface']:
    surface_dict[surface] = surface_dict.get(surface, 0) + 1

print(surface_dict)

#### Calculating surface ELO

In [ ]:
from collections import defaultdict

# Creates a nested defaultdict: elo_dict[player_id][surface] = ELO
elo_dict = defaultdict(lambda: defaultdict(lambda: 1500))

# Lists to store ELOs before the match
p1_surface_elo_before = []
p2_surface_elo_before = []

# Loop through df_subset row by row
for surface, p1, p2 in zip(df_subset['surface'], df_subset['p1_id'], df_subset['p2_id']):

    # Get each player's current ELO on this surface
    p1_elo = elo_dict[p1][surface]
    p2_elo = elo_dict[p2][surface]

    # Store ELOs before match
    p1_surface_elo_before.append(p1_elo)
    p2_surface_elo_before.append(p2_elo)

    # Calculate expected scores
    p1_expected = 1 / (1 + 10 ** ((p2_elo - p1_elo) / 400))
    p2_expected = 1 / (1 + 10 ** ((p1_elo - p2_elo) / 400))

    # Update ELOs assuming p1 wins
    K = 32
    elo_dict[p1][surface] = int(p1_elo + K * (1 - p1_expected))
    elo_dict[p2][surface] = int(p2_elo + K * (0 - p2_expected))

# Add columns to df_subset
df_subset['p1_surface_elo_before'] = p1_surface_elo_before
df_subset['p2_surface_elo_before'] = p2_surface_elo_before

#### Calculating Rolling Surface ELO

In [ ]:
df_subset['p1_surface_elo_rolling_last5'] = (df_subset.groupby('p1_id')['p1_surface_elo_before'].transform(lambda x: x.shift().rolling(5, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p2_surface_elo_rolling_last5'] = (df_subset.groupby('p2_id')['p2_surface_elo_before'].transform(lambda x: x.shift().rolling(5, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p1_surface_elo_rolling_last10'] = (df_subset.groupby('p1_id')['p1_surface_elo_before'].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p2_surface_elo_rolling_last10'] = (df_subset.groupby('p2_id')['p2_surface_elo_before'].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p1_surface_elo_rolling_last20'] = (df_subset.groupby('p1_id')['p1_surface_elo_before'].transform(lambda x: x.shift().rolling(20, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p2_surface_elo_rolling_last20'] = (df_subset.groupby('p2_id')['p2_surface_elo_before'].transform(lambda x: x.shift().rolling(20, min_periods=1).mean())).fillna(1500).astype(int)

#### Calculating Rolling Surface ELO difference

In [ ]:
# Create empty lists to store rolling ELO differences
p1_surface_elo_rolling_last5_diff_list = []
p2_surface_elo_rolling_last5_diff_list = []

p1_surface_elo_rolling_last10_diff_list = []
p2_surface_elo_rolling_last10_diff_list = []

p1_surface_elo_rolling_last20_diff_list = []
p2_surface_elo_rolling_last20_diff_list = []

# Loops over the last5 rolling ELO columns and calculates the players ELO difference
for p1, p2 in zip(df_subset['p1_surface_elo_rolling_last5'], df_subset['p2_surface_elo_rolling_last5']):
    p1_surface_elo_rolling_last5_diff = p1 - p2
    p2_surface_elo_rolling_last5_diff = p2 - p1

    # Adds calculated ELO differences to list
    p1_surface_elo_rolling_last5_diff_list.append(p1_surface_elo_rolling_last5_diff)
    p2_surface_elo_rolling_last5_diff_list.append(p2_surface_elo_rolling_last5_diff)

# Loops over the last10 rolling ELO columns and calculates the players ELO difference
for p1, p2 in zip(df_subset['p1_surface_elo_rolling_last10'], df_subset['p2_surface_elo_rolling_last10']):
    p1_surface_elo_rolling_last10_diff = p1 - p2
    p2_surface_elo_rolling_last10_diff = p2 - p1

    # Adds calculated ELO differences to list
    p1_surface_elo_rolling_last10_diff_list.append(p1_surface_elo_rolling_last10_diff)
    p2_surface_elo_rolling_last10_diff_list.append(p2_surface_elo_rolling_last10_diff)

# Loops over the last20 rolling ELO columns and calculates the players ELO difference
for p1, p2 in zip(df_subset['p1_surface_elo_rolling_last20'], df_subset['p2_surface_elo_rolling_last20']):
    p1_surface_elo_rolling_last20_diff = p1 - p2
    p2_surface_elo_rolling_last20_diff = p2 - p1

    # Adds calculated ELO differences to list
    p1_surface_elo_rolling_last20_diff_list.append(p1_surface_elo_rolling_last20_diff)
    p2_surface_elo_rolling_last20_diff_list.append(p2_surface_elo_rolling_last20_diff)


# Assign lists to new df columns
df_subset['p1_surface_elo_rolling_last5_diff_before'] = p1_surface_elo_rolling_last5_diff_list
df_subset['p2_surface_elo_rolling_last5_diff_before'] = p2_surface_elo_rolling_last5_diff_list
df_subset['p1_surface_elo_rolling_last10_diff_before'] = p1_surface_elo_rolling_last10_diff_list
df_subset['p2_surface_elo_rolling_last10_diff_before'] = p2_surface_elo_rolling_last10_diff_list
df_subset['p1_surface_elo_rolling_last20_diff_before'] = p1_surface_elo_rolling_last20_diff_list
df_subset['p2_surface_elo_rolling_last20_diff_before'] = p2_surface_elo_rolling_last20_diff_list

#### Calculating surface ELO difference

In [ ]:
# Creates empty lists to store surface ELO differences
p1_surface_elo_diff_list = []
p2_surface_elo_diff_list = []

# Loops over the surface ELO columns and calculates the surface ELO difference 
for p1, p2 in zip(df_subset['p1_surface_elo_before'], df_subset['p2_surface_elo_before']):
    p1_surface_elo_diff = p1 - p2
    p2_surface_elo_diff = p2 - p1

    # Adds calculates surface ELO difference to list
    p1_surface_elo_diff_list.append(p1_surface_elo_diff)
    p2_surface_elo_diff_list.append(p2_surface_elo_diff)

# Assign lists to new df columns
df_subset['p1_surface_elo_diff_before'] = p1_surface_elo_diff_list
df_subset['p2_surface_elo_diff_before'] = p2_surface_elo_diff_list

# Calculating Total Number of matches a Player Has Played

In [ ]:
from collections import defaultdict

# Dictionary to store the number of matches each player has played before
total_matches_dict = defaultdict(int)

# Empty lists to store total number of matches played by each player 
p1_total_matches_before = []
p2_total_matches_before = []

# Loops over player id columns and creates a key based on the players id
for p1, p2 in zip(df_subset['p1_id'], df_subset['p2_id']):
    total_matches_key1 = p1
    total_matches_key2 = p2

    # Get current total number of matches for each player before current game
    p1_total_matches = total_matches_dict[total_matches_key1]
    p2_total_matches = total_matches_dict[total_matches_key2]

    # Stores the total number of matches for each player before current game
    p1_total_matches_before.append(p1_total_matches)
    p2_total_matches_before.append(p2_total_matches)

    # Increment the match count for each player
    p1_total_matches_after = p1_total_matches + 1
    p2_total_matches_after = p2_total_matches + 1

    # Update the dictionary with the new total matches played after this match 
    total_matches_dict[total_matches_key1] = p1_total_matches_after
    total_matches_dict[total_matches_key2] = p2_total_matches_after

# Assign lists to new df columns
df_subset['p1_total_matches_played_before'] = p1_total_matches_before
df_subset['p2_total_matches_played_before'] = p2_total_matches_before

#### Calculate total matches player difference

In [ ]:
p1_total_career_matches_diff_list = []
p2_total_career_matches_diff_list = []

for p1, p2 in zip(df_subset['p1_total_matches_played_before'], df_subset['p2_total_matches_played_before']):
    p1_career_matches_diff = p1 - p2
    p2_career_matches_diff = p2 - p1

    p1_total_career_matches_diff_list.append(p1_career_matches_diff)
    p2_total_career_matches_diff_list.append(p2_career_matches_diff)

df_subset['p1_total_matches_played_before_diff'] = p1_total_career_matches_diff_list
df_subset['p2_total_matches_played_before_diff'] = p2_total_career_matches_diff_list

# Calculating Total Wins

In [ ]:
from collections import defaultdict

# Initialize dictionaries to track career wins and match outcomes
career_wins_dict = defaultdict(int)
career_wins_history_dict = defaultdict(list)

# Lists to store computed features for Dataframe
p1_career_wins_before = []
p2_career_wins_before = []

# Player win percentage in last 3 matches before current match
p1_career_wins_last3_pct_before = []
p2_career_wins_last3_pct_before = []

# Player win percentage in last 5 matches before current match
p1_career_wins_last5_pct_before = []
p2_career_wins_last5_pct_before = []

# Player win percentage in last 10 matches before current match
p1_career_wins_last10_pct_before = []
p2_career_wins_last10_pct_before = []

# Define time periods for recent win percentages 
x1 = 3
x2 = 5
x3 = 10

for p1, p2 in zip(df_subset['p1_id'], df_subset['p2_id']):
    # Store total wins before this match
    p1_career_wins_before.append(career_wins_dict[p1])
    p2_career_wins_before.append(career_wins_dict[p2])

    # Get last 3 outcomes
    p1_last3 = career_wins_history_dict[p1][-x1:]
    p2_last3 = career_wins_history_dict[p2][-x1:]

    # Get last 5 outcomes
    p1_last5 = career_wins_history_dict[p1][-x2:]
    p2_last5 = career_wins_history_dict[p2][-x2:]

    # Get last 10 outcomes
    p1_last10 = career_wins_history_dict[p1][-x3:]
    p2_last10 = career_wins_history_dict[p2][-x3:]

    # Compute recent win 3 match win percentage
    p1_last3_pct = round((sum(p1_last3) / x1) * 100, 1) if len(p1_last3) == x1 else 0
    p2_last3_pct = round((sum(p2_last3) / x1) * 100, 1) if len(p2_last3) == x1 else 0

    # Compute recent 5 match win percentage
    p1_last5_pct = round((sum(p1_last5) / x2) * 100, 1) if len(p1_last5) == x2 else 0
    p2_last5_pct = round((sum(p2_last5) / x2) * 100, 1) if len(p2_last5) == x2 else 0

    # Compute recent 10 match win percentage
    p1_last10_pct = round((sum(p1_last10) / x3) * 100, 1) if len(p1_last10) == x3 else 0
    p2_last10_pct = round((sum(p2_last10) / x3) * 100, 1) if len(p2_last10) == x3 else 0

    # Append percentages to respective lists
    p1_career_wins_last3_pct_before.append(p1_last3_pct)
    p2_career_wins_last3_pct_before.append(p2_last3_pct)

    p1_career_wins_last5_pct_before.append(p1_last5_pct)
    p2_career_wins_last5_pct_before.append(p2_last5_pct)

    p1_career_wins_last10_pct_before.append(p1_last10_pct)
    p2_career_wins_last10_pct_before.append(p2_last10_pct)

    
    # Update total wins
    career_wins_dict[p1] += 1

    # Update recent win/loss history
    career_wins_history_dict[p1].append(1)  # p1 won
    career_wins_history_dict[p2].append(0)  # p2 lost

# Assign to DataFrame
df_subset['p1_career_wins_before'] = p1_career_wins_before
df_subset['p2_career_wins_before'] = p2_career_wins_before
df_subset['p1_career_wins_last3_pct_before'] = p1_career_wins_last3_pct_before
df_subset['p2_career_wins_last3_pct_before'] = p2_career_wins_last3_pct_before
df_subset['p1_career_wins_last5_pct_before'] = p1_career_wins_last5_pct_before
df_subset['p2_career_wins_last5_pct_before'] = p2_career_wins_last5_pct_before
df_subset['p1_career_wins_last10_pct_before'] = p1_career_wins_last10_pct_before
df_subset['p2_career_wins_last10_pct_before'] = p2_career_wins_last10_pct_before

# Removing Anomalous Data after ELOs and Match Histories have been Calculated

In [ ]:
# Remove rows with NaN values for height, surface and age columns
df_subset = df_subset.dropna(subset=['p1_height'])
df_subset = df_subset.dropna(subset=['p2_height'])
df_subset = df_subset.dropna(subset=['surface'])
df_subset = df_subset.dropna(subset=['p1_age'])
df_subset = df_subset.dropna(subset=['p2_age'])

# Remove rows with unknown player handedness
df_subset = df_subset[(df_subset['p1_hand'] != 'U') & (df_subset['p2_hand'] != 'U') & (df_subset['p2_hand'] != 'A')]

#List of anomalous players to remove
anom_heights_list = ['Jorge Brian Panta Herreros',
'Johannes Ingildsen',
'Viacheslav Bielinskyi',
'Kooros Ghasemi',
'Alexander Stater',                     
'William Grant',
'Ilija Vucic',
'Andrew Rogers'
]

# remove anomalous players
mask = df_subset['p1_name'].isin(anom_heights_list) | df_subset['p2_name'].isin(anom_heights_list)
df_subset = df_subset.drop(df_subset[mask].index)

###### Checking for any NaN values or anomolous data

In [ ]:
# Displayes number of NaN values for each column
df_subset.isna().sum()

In [ ]:
df_subset.describe()

# Reset Indexing

In [ ]:
df_subset = df_subset.reset_index(drop=True)

# Creating Target Feature

###### In the data I am using, the winner of the match is always player 1. To model this data, the winner must be randomly player 1 or player 2 and a new 'result' feature needs to be added 

In [ ]:
import numpy as np

# Make a copy so original data isn't affected
df_balanced = df_subset.copy()

# Add a result column and make the value equal to 1 because player 1 always wins
df_balanced["result"] = 1

# Randomly choose half of the rows to swap player1 and player2
swap_mask = np.random.rand(len(df_subset)) < 0.5

# Swap player-related columns where swap_mask is True
cols_to_swap = [
    ('p1_id', 'p2_id'),
    ('p1_hand', 'p2_hand'),
    ('p1_name', 'p2_name'),
    ('p1_age', 'p2_age'),
    ('p1_height', 'p2_height'),
    ('p1_age_diff', 'p2_age_diff'),
    ('p1_height_diff', 'p2_height_diff'),
    ('p1_h2h_wins', 'p2_h2h_wins'),
    ('p1_h2h_wins_before_total_diff', 'p2_h2h_wins_before_total_diff'),
    ('p1_h2h_wins_before_last1_diff', 'p2_h2h_wins_before_last1_diff'),
    ('p1_h2h_wins_before_last2_diff', 'p2_h2h_wins_before_last2_diff'),
    ('p1_h2h_wins_before_last3_diff', 'p2_h2h_wins_before_last3_diff'),
    ('p1_h2h_wins_before_last4_diff', 'p2_h2h_wins_before_last4_diff'),
    ('p1_h2h_wins_before_last5_diff', 'p2_h2h_wins_before_last5_diff'),
    ('p1_h2h_wins_before_last10_diff', 'p2_h2h_wins_before_last10_diff'),
    ('p1_elo_before', 'p2_elo_before'),
    ('p1_elo_diff_before', 'p2_elo_diff_before'),
    ('p1_surface_elo_before', 'p2_surface_elo_before'),
    ('p1_surface_elo_diff_before', 'p2_surface_elo_diff_before'),
    ('p1_total_matches_played_before', 'p2_total_matches_played_before'),
    ('p1_total_matches_played_before_diff', 'p2_total_matches_played_before_diff'),
    ('p1_career_wins_before', 'p2_career_wins_before'),
    ('p1_career_wins_last3_pct_before', 'p2_career_wins_last3_pct_before'),
    ('p1_career_wins_last5_pct_before', 'p2_career_wins_last5_pct_before'),
    ('p1_career_wins_last10_pct_before', 'p2_career_wins_last10_pct_before'),
    ('p1_elo_rolling_last5', 'p2_elo_rolling_last5'),
    ('p1_elo_rolling_last10', 'p2_elo_rolling_last10'),
    ('p1_elo_rolling_last20', 'p2_elo_rolling_last20'),
    ('p1_elo_rolling_last5_diff_before', 'p2_elo_rolling_last5_diff_before'),
    ('p1_elo_rolling_last10_diff_before', 'p2_elo_rolling_last10_diff_before'),
    ('p1_elo_rolling_last20_diff_before', 'p2_elo_rolling_last20_diff_before'),
    ('p1_surface_elo_rolling_last5', 'p2_surface_elo_rolling_last5'),
    ('p1_surface_elo_rolling_last10', 'p2_surface_elo_rolling_last10'),
    ('p1_surface_elo_rolling_last20', 'p2_surface_elo_rolling_last20'),
    ('p1_surface_elo_rolling_last5_diff_before', 'p2_surface_elo_rolling_last5_diff_before'),
    ('p1_surface_elo_rolling_last10_diff_before', 'p2_surface_elo_rolling_last10_diff_before'),
    ('p1_surface_elo_rolling_last20_diff_before', 'p2_surface_elo_rolling_last20_diff_before')
]

# Loop through each pair of columns and swap values for the selected rows
for col1, col2 in cols_to_swap:
    temp = df_balanced.loc[swap_mask, col1].copy()
    df_balanced.loc[swap_mask, col1] = df_balanced.loc[swap_mask, col2]
    df_balanced.loc[swap_mask, col2] = temp

# Update the result column to 0 where players were swapped
df_balanced.loc[swap_mask, "result"] = 0

# Removing Redundant Features

In [ ]:
redundant_features = [
    'tournament_name',
    'p1_id',
    'p2_id',
    'p2_age_diff',
    'p2_height_diff',
    'p2_h2h_wins_before_total_diff',
    'p2_h2h_wins_before_last1_diff',
    'p2_h2h_wins_before_last2_diff',
    'p2_h2h_wins_before_last3_diff',
    'p2_h2h_wins_before_last4_diff',
    'p2_h2h_wins_before_last5_diff',
    'p2_h2h_wins_before_last10_diff',
    'p2_elo_diff_before',
    'p2_surface_elo_diff_before',
    'p2_total_matches_played_before_diff',
    'p2_career_wins_last3_pct_before',
    'p2_career_wins_last5_pct_before',
    'p2_career_wins_last10_pct_before'
]

# Drop them from the DataFrame
df_balanced = df_balanced.drop(redundant_features, axis=1)

# Encoding Players Names

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Collect all players names 
all_players = pd.concat([df_balanced['p1_name'], df_balanced['p2_name']])

# Fit a LabelEncoder on all names
le_players = LabelEncoder()
le_players.fit(all_players)

# Transform player name columns to player id columns
df_balanced['p1_id'] = le_players.transform(df_balanced['p1_name'])
df_balanced['p2_id'] = le_players.transform(df_balanced['p2_name'])

# Encode player hand
le_hand = {'R': 0, 'L': 1}
df_balanced['p1_hand_id'] = df_balanced['p1_hand'].map(le_hand)
df_balanced['p2_hand_id'] = df_balanced['p2_hand'].map(le_hand)
df_balanced['p1_hand_id'] = df_balanced['p1_hand_id'].astype(int)
df_balanced['p2_hand_id'] = df_balanced['p2_hand_id'].astype(int)

# Encode surface
le_surface = LabelEncoder()
df_balanced['surface_id'] = le_surface.fit_transform(df_balanced['surface'])

# Encode tournament level
le_tournament_level = LabelEncoder()
df_balanced['tournament_level_id'] = le_tournament_level.fit_transform(df_balanced['tournament_level'])

# Reordering Column Names

In [ ]:
cols_new_order = [
    'tournament_date',
    'tournament_level_id',
    'surface_id',
    'p1_id',
    'p1_hand_id',
    'p2_id',
    'p2_hand_id',
    'p1_age_diff',
    'p1_height_diff',
    'p1_h2h_wins_before_total_diff',
    'p1_elo_diff_before',
    'p1_elo_rolling_last5_diff_before',
    'p1_elo_rolling_last10_diff_before',
    'p1_elo_rolling_last20_diff_before',
    'p1_surface_elo_diff_before',
    'p1_surface_elo_rolling_last5_diff_before',
    'p1_surface_elo_rolling_last10_diff_before',
    'p1_surface_elo_rolling_last20_diff_before',
    'p1_total_matches_played_before_diff',
    'result'
]
df_balanced = df_balanced[cols_new_order].copy()

# Saving Dataframe as a new .csv file

In [ ]:
df_balanced.info()

In [ ]:
df_balanced.to_csv('feature_engineered_tennis_data.csv', index = False)